In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import math 
import copy
import random
import argparse


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
def load_data(dataset_name, K):
    if dataset_name == "statlog":
        ##statlog
        __context_size__ = 8
        __arm_size__ = 7
        fr = open("shuttle.trn",'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K+=1
            aaa = line.split("\n")
            aaa = aaa[0].split(" ")
            temp = np.int64(aaa[9])-1
            context = np.double(aaa[1:9])
            context = context/np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)

    elif dataset_name == "magic":
        ####magic gamma

        __context_size__ = 10
        __arm_size__ = 2
        fr = open("letter.data", 'r+')

        K = 0
        CONTEXT = []
        REWARD = []

        fr.close()
        fr = open("magic04.data", 'r+')
        for line in fr:

            context = []
            aaa = line.split(",")
            context = np.double(aaa[0:__context_size__])
            context = context / np.linalg.norm(context)
            K += 1
            CONTEXT.append(np.array(context))
            __context_size__ = len(context)
            reward = np.zeros(__arm_size__)
            if aaa[10] == 'g\n':
                reward[0] = 1
            else:
                reward[1] = 1
            REWARD.append(reward)

        fr.close()
    elif dataset_name == "covertype":
        ##For covertypr
        __context_size__ = 14
        __arm_size__ = 7
        fr = open("covtype.data", 'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K += 1
            aaa = line.split(",")
            __context_size__ = len(aaa) - 1
            temp = int(aaa[__context_size__]) - 1
            context = np.double(aaa[0:__context_size__])
            context = context / np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)
    else:
        assert('dataset not avaiable')

    TEMP_CONTEXT = []
    TEMP_REWARD = []
    sli = np.random.permutation(K)
    for i in range(K):
        TEMP_CONTEXT.append(CONTEXT[sli[i]])
        TEMP_REWARD.append(REWARD[sli[i]])
    CONTEXT = TEMP_CONTEXT
    REWARD = TEMP_REWARD

    return __context_size__, __arm_size__, CONTEXT, REWARD, sli

In [3]:
# define the hyperparameters
input_size = 15
hidden_size = 50
output_size = 15
num_layers = 10
arm_size = 7
beta = 0.1
lambd = 1
H = 100

In [4]:
# initialize the parameters

theta = np.random.randn(output_size, 1) / np.sqrt(output_size)
theta = torch.tensor(theta, dtype = torch.float32)

LAMBDA = lambd * torch.eye(output_size, dtype=torch.float32)
bb = torch.zeros(LAMBDA.size()[0], dtype=torch.float).view(-1, 1)

In [5]:
# create the model 
import modeldefine
model = modeldefine.Model(input_size, hidden_size, output_size, num_layers).to(device)

tensor([15, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 15], dtype=torch.int32)


In [6]:
def TRANS(c, a, arm_size):
    #### transfer an array context + action to new context with dimension 2*(__context__ + __armsize__)
    dim = len(c)
    action = np.zeros(arm_size)
    action[a] = 1
    c_final = np.append(c, action)
    c_final = torch.from_numpy(c_final).to(device)
    c_final = c_final.view((len(c_final), 1))
    c_final = c_final.repeat(2, 1)
    c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)
    return c_final

def UCB(A, phi):
    #### ucb term
    phi = phi.view(-1,1)
    try:
        tmp, LU = torch.linalg.solve(phi,A)
    except:
        A = A.detach().numpy()
        phi2 = phi.detach().numpy()
        tmp = torch.Tensor(np.linalg.solve(A, phi2))

    return torch.sqrt(torch.matmul(torch.transpose(phi,1,0), tmp))

def get_action(model, context_info, random_indicator):
    ucb = [] 
    bphi = []
    for i in range(arm_size):
        temp = TRANS(context_info, i, arm_size).to(device)
        bphi.append(temp.cpu())
        feature = model(temp).cpu()

        first_item = torch.mm(feature, theta)
        ucb_item = beta * UCB(LAMBDA, feature)
        ucb.append(first_item + ucb_item)
    
    if random_indicator == 0:
        return ucb.index(max(ucb)), bphi[ucb.index(max(ucb))] 
    else:
        num =  np.random.randint(0, arm_size)
        return num, bphi[num]
    
def update_lambda_bb(lambd, bb, context_info, action,reward):
    temp = TRANS(context_info, action, arm_size).to(device)
    feature = model(temp).cpu()
    templambda = lambd + torch.mm(feature.t(), feature)
    tempbb = bb + reward * feature.t()
    return templambda, tempbb

def update_theta(lambd, bb):
    return torch.linalg.solve(lambd, bb)

In [30]:
THETA_action = []
CONTEXT_action = []
REWARD_action = []
context_size, arm_size, CONTEXT_list, REWARD, sli = load_data("statlog", 15000)

for t in range(0,100):
    context = CONTEXT_list[t]
    if t <= 21:
        action, c_act = get_action(model, context, 1)
    else: 
        action, c_act = get_action(model, context, 0) 

    # get the reward
    reward = REWARD[t][action] 

    # construct the training set for neural network
    if t % H == 0:
        CONTEXT_action = []
        REWARD_action = []
        CONTEXT_action = c_act
        REWARD_action = torch.tensor([reward],device= device, dtype=torch.float32)
    else:
        CONTEXT_action = torch.cat((CONTEXT_action, c_act), 0)
        REWARD_action = torch.cat((REWARD_action, torch.tensor([reward],device= device, dtype=torch.float32)), 0)

    # update lambda, bb and theta
    LAMBDA, bb = update_lambda_bb(LAMBDA,bb,context, action, reward)
    theta = update_theta(LAMBDA, bb)

    if t % H == 0:
        THETA_action = []
        THETA_action = theta.view(1,-1)
    else: 
        THETA_action = torch.cat((THETA_action, theta.view(1,-1)), 0)
    
    # update the model
    


C:\Users\杜骏晔\AppData\Local\Temp\ipykernel_18920\3790411606.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)


In [38]:
torch.matmul(model(CONTEXT_action.to(device)), THETA_action.t().to(device))

tensor([[0.6096, 0.6071, 0.6098,  ..., 0.5341, 0.5318, 0.5327],
        [0.3355, 0.3134, 0.3158,  ..., 0.2015, 0.2005, 0.2014],
        [0.6991, 0.6964, 0.6995,  ..., 0.6919, 0.6880, 0.6894],
        ...,
        [0.7442, 0.7437, 0.7467,  ..., 0.7426, 0.7381, 0.7396],
        [0.7561, 0.7548, 0.7578,  ..., 0.7594, 0.7541, 0.7557],
        [0.7296, 0.7275, 0.7306,  ..., 0.7319, 0.7274, 0.7290]],
       device='cuda:0', grad_fn=<MmBackward0>)

In [39]:
jk = model(CONTEXT_action.to(device))
jk

tensor([[0.7294, 0.3440, 1.2253,  ..., 0.1113, 1.0948, 3.2113],
        [0.6865, 0.2541, 1.0781,  ..., 0.2497, 1.1147, 3.0314],
        [0.7134, 0.2894, 1.1854,  ..., 0.1431, 1.0804, 3.1730],
        ...,
        [0.7306, 0.3033, 1.1899,  ..., 0.1364, 1.0617, 3.1882],
        [0.7330, 0.2982, 1.1776,  ..., 0.1660, 1.0597, 3.1886],
        [0.7246, 0.2967, 1.1761,  ..., 0.1522, 1.0689, 3.1739]],
       device='cuda:0', grad_fn=<MulBackward0>)

In [47]:
bc = THETA_action.t()
bc

tensor([[ 0.1527,  0.1527,  0.1510,  ..., -0.0243, -0.0291, -0.0284],
        [-0.0975, -0.0873, -0.0898,  ..., -0.3149, -0.3140, -0.3144],
        [ 0.0526,  0.0732,  0.0740,  ...,  0.1300,  0.1345,  0.1335],
        ...,
        [ 0.0551,  0.0229,  0.0228,  ...,  0.1395,  0.1244,  0.1263],
        [-0.2532, -0.2693, -0.2671,  ..., -0.2717, -0.2651, -0.2649],
        [ 0.3220,  0.3323,  0.3330,  ...,  0.3820,  0.3773,  0.3784]],
       grad_fn=<TBackward0>)

In [56]:
aaaa = jk @ bc.to(device) 

In [55]:
jk.cpu().detach().numpy() @ bc.cpu().detach().numpy()

array([[0.6095652 , 0.60713196, 0.6097884 , ..., 0.534112  , 0.53178   ,
        0.53269136],
       [0.3354706 , 0.31342348, 0.3158491 , ..., 0.20150429, 0.20051885,
        0.20135725],
       [0.69914514, 0.69643676, 0.6995219 , ..., 0.69186056, 0.687984  ,
        0.6894487 ],
       ...,
       [0.7442129 , 0.74369943, 0.7466913 , ..., 0.7426453 , 0.7381208 ,
        0.73962396],
       [0.75612366, 0.75478244, 0.75779575, ..., 0.7593875 , 0.7541054 ,
        0.75571346],
       [0.7296291 , 0.72749746, 0.7305664 , ..., 0.7318702 , 0.72736484,
        0.7289618 ]], dtype=float32)

In [60]:
torch.sum(jk * bc.t().to(device), dim=1)

tensor([0.6096, 0.3134, 0.6995, 0.4725, 0.2691, 0.4685, 0.4171, 0.5308, 0.3941,
        0.4988, 0.2112, 0.2002, 0.4618, 0.3680, 0.2457, 0.4959, 0.4918, 0.7236,
        0.4593, 0.2610, 0.5125, 0.5106, 0.7023, 0.6129, 0.7500, 0.7025, 0.7086,
        0.6630, 0.6550, 0.7228, 0.7010, 0.6387, 0.7227, 0.7227, 0.6904, 0.7198,
        0.7296, 0.7233, 0.6725, 0.6428, 0.7452, 0.7251, 0.6847, 0.7258, 0.6833,
        0.6858, 0.7017, 0.6346, 0.7075, 0.7049, 0.7283, 0.6567, 0.7439, 0.7065,
        0.7108, 0.7248, 0.7069, 0.7028, 0.6360, 0.6869, 0.7532, 0.6796, 0.7015,
        0.6565, 0.7382, 0.7257, 0.6649, 0.7039, 0.7147, 0.7511, 0.7102, 0.7257,
        0.7308, 0.7492, 0.7214, 0.7659, 0.7252, 0.6869, 0.7547, 0.6375, 0.7627,
        0.7175, 0.7017, 0.7427, 0.7268, 0.7218, 0.6741, 0.7183, 0.7063, 0.6952,
        0.6870, 0.6662, 0.7296, 0.6873, 0.7126, 0.7271, 0.7663, 0.7426, 0.7541,
        0.7290], device='cuda:0', grad_fn=<SumBackward1>)

In [62]:
REWARD_action

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        0., 0., 1., 1., 1., 1., 1., 1., 0., 1.], device='cuda:0')